In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

from utils_dataset2mne import data2mne
from utils_dataset2mne import create_name
from utils_dataset2mne import apply_ASR
from utils_dataset2mne import plot_TF_M
from utils_dataset2mne import plot_TF_M_stat
from utils_dataset2mne import plot_ERDs

mne.set_log_level(verbose='WARNING')

%matplotlib qt

In [2]:
report = mne.Report(title="Report Time-Frequency - Motor Imagery Dataset GigaScience")

#### Creating Epochs

In [3]:
subjects = list(range(1, 53))

for subject in subjects:
    print('Processing Subject s' + str(subject))

    fname_mat = create_name(subject,'EEG_Data')

    # Convert .mat data to a raw object of MNE-python
    raw = data2mne(fname_mat, scale=1e-8)

    # Event ID definition
    event_id = {'Blink':1,'Up/Down':2,'Left/Right':3,'Jaw':4,'HeadMov':5, 'Resting':6, 'ME_Right':7, 'ME_Left':8, 'MI_Right':9, 'MI_Left':10}

    # Read events from Stimuli channel
    events = mne.find_events(raw, stim_channel='Stim', verbose=True,initial_event=True)

    # Plot Raw Data
    # raw.plot(events=events,event_id=event_id)

    # Only imaginary events
    event_id_MI = {'MI_Right':9, 'MI_Left':10}
    events_MI = events[events[:,2] > 8]

    # Change the custom montage to standard positions for better visualization
    biosemi64_montage = mne.channels.make_standard_montage('biosemi64')
    raw.set_montage(biosemi64_montage)

    # Create Epochs
    epochs = mne.Epochs(raw, events_MI, event_id_MI, tmin=-1.95, tmax=4.95, picks=('eeg'), baseline=None, preload=True)
    epochs.filter(0.5,None)

    # Common Average Reference as a projection
    epochs.set_eeg_reference(ref_channels='average',projection=True)

    # Plot Raw Data only with imaginary events
    # raw.plot(events=events_MI,event_id=event_id_M)

    # Electrode position 2D
    # raw.plot_sensors(kind='topomap',ch_type='eeg', show_names=True, sphere='eeglab')

    # Add epochs to the report
    report.add_epochs(epochs=epochs, title='Epochs - Highpass 0.5Hz', tags='s'+str(subject))

    # Prepare Epochs for TF
    channels = ['C3','C4']
    epochs_TF_raw = epochs.copy().apply_proj().pick_channels(channels)
    # epochs_TF_ASR = epochs_ASR.copy().apply_proj().pick_channels(channels)

    # Compute TF Maps - Wavelet
    freqs = np.arange(2, 35.5, 0.5)
    n_cycles = freqs / 2.0  # different number of cycle per frequency
    TF_raw_wav = mne.time_frequency.tfr_morlet(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
    # TF_ASR_wav = mne.time_frequency.tfr_morlet(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

    # Compute TF Maps - Multitaper
    TF_raw_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_raw['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)
    # TF_ASR_mtaper = mne.time_frequency.tfr_multitaper(epochs_TF_ASR['MI_Left','MI_Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3, n_jobs=4, average=False)

    # Plot TF Maps - Wavelet
    event_ids = ['MI_Right','MI_Left'] # events 
    kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, fmin=None, fmax=None, cmap='jet')
    figs1 = plot_TF_M(TF_raw_wav, event_ids, **kwargs, comment='raw - Wavelet - s' + str(subject), reporting=True)
    # figs2 = plot_TF_M(TF_ASR_wav, event_ids, **kwargs, comment='ASR - Wavelet - s' + str(subject), reporting=True)

    # Add to report
    for fig in figs1:
        report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
    # for fig in figs2:
    #     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))

    # Plot TF Maps - Multitaper
    figs1 = plot_TF_M(TF_raw_mtaper, event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject), reporting=True)
    # figs2 = plot_TF_M(TF_ASR_mtaper, event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject), reporting=True)

    # Add to report
    for fig in figs1:
        report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
    # for fig in figs2:
    #     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))

    # Plot ERD per frequency bands
    channels = ['C3','C4']
    fig = plot_ERDs(TF_raw_mtaper,channels, comment='raw - Multitapers - s' + str(subject), reporting=True)

    # Add to report
    fig.savefig('temp.png')
    report.add_image(image='temp.png', title='ERDS mu/alpha & beta', section='TF - ERDs Maps', tags='s'+str(subject))

    # Plot ERD maps - Multitaper - Statistically significant
    # kwargs = dict(baseline=(-1,0), mode='percent', vmin=-1.2, vmax=1.2, cmap='jet')
    # figs1 = plot_TF_M_stat(TF_raw_mtaper.copy(), event_ids, **kwargs, comment='raw - Multitaper - s' + str(subject), reporting=True)
    # figs2 = plot_TF_M_stat(TF_ASR_mtaper.copy(), event_ids, **kwargs, comment='ASR - Multitaper - s' + str(subject), reporting=True)

    # Add to report
    # for fig in figs1:
    #     report.add_figure(fig=fig, title='Statistically Significant ERDS', section='TF - ERDs Maps', tags='s'+str(subject))
    # for fig in figs2:
    #     report.add_figure(fig=fig, title='ERDS', section='TF - ERDs Maps', tags='s'+str(subject))

    # Save Report
    report.save("Report - GigaScience.hdf5", overwrite=True)
    report.save("Report - GigaScience.html", overwrite=True)

    plt.close("all")



Processing Subject s1
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s2
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s3
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s4
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s5
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s6
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s7
286 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s8
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s9
286 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s10
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s11
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Subject s12
246 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10]
Processing Su

In [11]:
epochs["MI_Right"].copy().apply_baseline().plot_image(picks=["C3", "C4"],vmin=-30, vmax=30)
epochs["MI_Left"].copy().apply_baseline().plot_image(picks=["C3", "C4"],vmin=-30, vmax=30)

[<Figure size 640x480 with 4 Axes>, <Figure size 640x480 with 4 Axes>]